In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from itertools import tee

In [2]:
df = pd.read_csv("Clusters+vert_cercanos.csv", delimiter=",")

In [3]:
df_seguridad_bomberos = pd.read_csv("seguridad_bomberos_oficiales.csv", delimiter=";")

In [4]:
df_seguridad_bomberos

,Id,Latitud_seguridad_bomberos,Longitud_seguridad_bomberos
0,1,4.148923,-73.635258


In [6]:
array_id=df_seguridad_bomberos.Id.values

In [7]:
df2=df.copy(deep=True)
for col_name in array_id:
    df2[col_name] = None

In [23]:
df2

,Unnamed: 0,x,y,index_vertice_cercano,LATITUD,LONGITUD,distancia_h,1
0,0,4.124433,-73.627487,33483,4.124,-73.627545,0.048541,"[655, 4158]"
1,1,4.118694,-73.563922,37514,4.119,-73.563647,0.045658,"[1244, 10265]"
2,2,4.075202,-73.672954,52077,4.075,-73.672891,0.023477,"[1186, 10322]"
3,3,4.153048,-73.599674,11110,4.153,-73.599651,0.005913,"[944, 4650]"
4,4,4.148361,-73.613292,14637,4.148,-73.613417,0.042406,"[642, 3008]"
...,...,...,...,...,...,...,...,...
195,195,4.158273,-73.651119,7341,4.158,-73.651163,0.030774,"[610, 3184]"
196,196,4.103017,-73.659508,45497,4.103,-73.659841,0.036985,"[775, 6375]"
197,197,4.155809,-73.632643,8546,4.156,-73.632766,0.025231,"[337, 1131]"
198,198,4.136147,-73.609554,23612,4.136,-73.609601,0.017196,"[758, 4244]"


In [1]:
#Se quita por motivos de seguridad
key_API_google=""

In [10]:
#aca origenes son destinos y destinos son origenes por que estamos midiendo cuanto se demora la policia en llegar allá

In [12]:
origenes = [row['Latitud_seguridad_bomberos'].astype(str)+"%2C"+row['Longitud_seguridad_bomberos'].astype(str) for index, row in df_seguridad_bomberos.iterrows()]

In [13]:
destinos = [row['LATITUD'].astype(str)+"%2C"+row['LONGITUD'].astype(str) for index, row in df.iterrows()]

In [14]:
todas_solicitudes=[]
for i in range(len(origenes)):
    solicitudes=[]
    for j in range(len(destinos)):
        solicitud="https://maps.googleapis.com/maps/api/distancematrix/json?destinations="+destinos[j]+"&origins="+origenes[i]+"&key="+key_API_google
        solicitudes.append(solicitud)
    todas_solicitudes.append(solicitudes)

In [15]:
todas_solicitudes2 = pd.DataFrame(todas_solicitudes)

In [16]:
todas_solicitudes2

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...,https://maps.googleapis.com/maps/api/distancem...


In [17]:
transposed_todas_solicitudes2 = todas_solicitudes2.transpose()

In [19]:
def solicitud_maps(url):
    try:
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        x = response.json()
        drive_time_seconds = x['rows'][0]['elements'][0]['duration']['value']
        distance_meters = x['rows'][0]['elements'][0]['distance']['value']
        return [drive_time_seconds, distance_meters]
    except KeyError:
        return 0

In [22]:
df2[1] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[0]), axis=1)

In [24]:
df3=df2.copy(deep=True)
df3 = df3.drop(array_id, axis=1)
for col in array_id:
    df3[col] = df2[col].apply(lambda x: x[0])

df3.to_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SEGURIDAD_BOMBEROS.csv",index=True)

In [25]:
df4=df2.copy(deep=True)
df4 = df4.drop(array_id, axis=1)
for col in array_id:
    df4[col] = df2[col].apply(lambda x: x[1])
    
df4.to_csv("DISTANCIAS_METROS_CLUSTERS_SEGURIDAD_BOMBEROS.csv",index=True)